In [28]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
import main.packages.unchain_chain as chain
import main.packages.mine_generic as mine_g
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
import re

I want to compare based on model:
- take category
- take model
- take rmse


In [29]:
date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')


In [30]:
cat_short = 'food'
forecast_df_cat = pd.read_csv(f"data/forecast_results/{cat_short}_forecast.csv")

forecast_df_cat.set_index(date_range, inplace=True)

return_cols = [col for col in forecast_df_cat.columns if cat_short in col]
forecast_df_cat = forecast_df_cat.loc[:, return_cols]

yoy_real = pd.read_csv(f"data/preprocessed/{cat_short}_yoy_infl.csv", parse_dates=True, index_col='date')
yoy_real_test = yoy_real[(yoy_real.index > mine_g.train_test_split_date) & (yoy_real.index <= mine_g.max_X_date)]


In [39]:
rmse = []

for h in [1, 2, 3]:
    h_cols = []

    for col in forecast_df_cat.columns:
        if col.endswith(str(h)):
            h_cols.append(col)
    forecast_df_h = forecast_df_cat[h_cols]
    
    for col in forecast_df_h.columns:
        rmse_here = np.sqrt(mean_squared_error(forecast_df_h[col], yoy_real_test))
        rmse.append(rmse_here)


def get_first_part(col_name, separator='_h'):
    return col_name.split(separator)[0]

new_col_names = [get_first_part(col) for col in forecast_df_h.columns]


rmse = pd.DataFrame(rmse)
rmse = pd.DataFrame(rmse.values.reshape(3, 5), columns=new_col_names, index=[f'h_{h}' for h in [1, 2, 3]])
rmse.to_csv(f"data/report_rmse/{cat_short}.csv")

---------

In [40]:
cat_short = 'energy'
forecast_df_cat = pd.read_csv(f"data/forecast_results/{cat_short}_forecast.csv")

forecast_df_cat.set_index(date_range, inplace=True)

return_cols = [col for col in forecast_df_cat.columns if cat_short in col]
forecast_df_cat = forecast_df_cat.loc[:, return_cols]

yoy_real = pd.read_csv(f"data/preprocessed/{cat_short}_yoy_infl.csv", parse_dates=True, index_col='date')
yoy_real_test = yoy_real[(yoy_real.index > mine_g.train_test_split_date) & (yoy_real.index <= mine_g.max_X_date)]


In [41]:
rmse = []

for h in [1, 2, 3]:
    h_cols = []

    for col in forecast_df_cat.columns:
        if col.endswith(str(h)):
            h_cols.append(col)
    forecast_df_h = forecast_df_cat[h_cols]
    
    for col in forecast_df_h.columns:
        rmse_here = np.sqrt(mean_squared_error(forecast_df_h[col], yoy_real_test))
        rmse.append(rmse_here)


def get_first_part(col_name, separator='_h'):
    return col_name.split(separator)[0]

new_col_names = [get_first_part(col) for col in forecast_df_h.columns]
rmse = pd.DataFrame(rmse)
rmse = pd.DataFrame(rmse.values.reshape(3, 5), columns=new_col_names, index=[f'h_{h}' for h in [1, 2, 3]])

rmse.to_csv(f"data/report_rmse/{cat_short}.csv")

,prc_energy,ridge_energy,lasso_energy,xgb_energy,ols_energy
h_1,3.488918,3.835047,3.712284,4.766152,3.331470
h_2,4.993943,5.372019,5.168972,5.722806,4.944986
h_3,6.039753,6.069338,5.913817,6.657155,6.269531


--------

In [43]:
cat_short = 'neig'
forecast_df_cat = pd.read_csv(f"data/forecast_results/{cat_short}_forecast.csv")

forecast_df_cat.set_index(date_range, inplace=True)

return_cols = [col for col in forecast_df_cat.columns if cat_short in col]
forecast_df_cat = forecast_df_cat.loc[:, return_cols]

yoy_real = pd.read_csv(f"data/preprocessed/{cat_short}_yoy_infl.csv", parse_dates=True, index_col='date')
yoy_real_test = yoy_real[(yoy_real.index > mine_g.train_test_split_date) & (yoy_real.index <= mine_g.max_X_date)]

rmse = []

for h in [1, 2, 3]:
    h_cols = []

    for col in forecast_df_cat.columns:
        if col.endswith(str(h)):
            h_cols.append(col)
    forecast_df_h = forecast_df_cat[h_cols]
    
    for col in forecast_df_h.columns:
        rmse_here = np.sqrt(mean_squared_error(forecast_df_h[col], yoy_real_test))
        rmse.append(rmse_here)


def get_first_part(col_name, separator='_h'):
    return col_name.split(separator)[0]

new_col_names = [get_first_part(col) for col in forecast_df_h.columns]
rmse = pd.DataFrame(rmse)
rmse = pd.DataFrame(rmse.values.reshape(3, 5), columns=new_col_names, index=[f'h_{h}' for h in [1, 2, 3]])

rmse.to_csv(f"data/report_rmse/{cat_short}.csv")

-----

In [44]:
cat_short = 'services'
forecast_df_cat = pd.read_csv(f"data/forecast_results/{cat_short}_forecast.csv")

forecast_df_cat.set_index(date_range, inplace=True)

return_cols = [col for col in forecast_df_cat.columns if cat_short in col]
forecast_df_cat = forecast_df_cat.loc[:, return_cols]

yoy_real = pd.read_csv(f"data/preprocessed/{cat_short}_yoy_infl.csv", parse_dates=True, index_col='date')
yoy_real_test = yoy_real[(yoy_real.index > mine_g.train_test_split_date) & (yoy_real.index <= mine_g.max_X_date)]

rmse = []

for h in [1, 2, 3]:
    h_cols = []

    for col in forecast_df_cat.columns:
        if col.endswith(str(h)):
            h_cols.append(col)
    forecast_df_h = forecast_df_cat[h_cols]
    
    for col in forecast_df_h.columns:
        rmse_here = np.sqrt(mean_squared_error(forecast_df_h[col], yoy_real_test))
        rmse.append(rmse_here)


def get_first_part(col_name, separator='_h'):
    return col_name.split(separator)[0]

new_col_names = [get_first_part(col) for col in forecast_df_h.columns]
rmse = pd.DataFrame(rmse)
rmse = pd.DataFrame(rmse.values.reshape(3, 5), columns=new_col_names, index=[f'h_{h}' for h in [1, 2, 3]])

rmse.to_csv(f"data/report_rmse/{cat_short}.csv")